## American Sign Language detection using Handtracking

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import mediapipe as mp
import os
import tkinter as tk
from sklearn.metrics import classification_report

# Create the CNN model
def create_cnn_model(input_shape=(64, 64, 3), num_classes=29):  # Example for ASL letters
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model using data from directories
def train_model(train_dir, test_dir):
    # Data augmentation and rescaling
    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Load the images from the folder path
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    print(f"Train samples: {train_generator.samples}")
    print(f"Test samples: {test_generator.samples}")

    model = create_cnn_model(input_shape=(64, 64, 3), num_classes=train_generator.num_classes)

    model.fit(train_generator, epochs=2, verbose=1)

    if test_generator.samples > 0:
        test_loss, test_acc = model.evaluate(test_generator)
        print(f"Test accuracy: {test_acc * 100:.2f}%")
    else:
        print("No test samples available for evaluation.")

    return model, list(test_generator.class_indices.keys())

# Hand detection and prediction function
def hand_tracking(model, class_names):
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)
    predicted_letter = "No Hand Detected"

    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8) as hands:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Extract the hand landmarks (edges)
                    hand_landmarks_array = []
                    for lm in hand_landmarks.landmark:
                        hand_landmarks_array.append([int(lm.x * image.shape[1]), int(lm.y * image.shape[0])])

                    # Create edges for the hand
                    hand_landmarks_array = np.array(hand_landmarks_array)
                    x, y, w, h = cv2.boundingRect(hand_landmarks_array)
                    hand_roi = image[y:y + h, x:x + w]

                    # Preprocess hand ROI for the model
                    if hand_roi.size == 0:
                        continue
                    hand_roi = cv2.resize(hand_roi, (64, 64)) / 255.0
                    hand_roi = np.expand_dims(hand_roi, axis=0)

                    # Predict the letter
                    pred = model.predict(hand_roi)
                    predicted_letter = class_names[np.argmax(pred)]

            # Show the predicted letter on screen
            cv2.putText(image, f"Predicted: {predicted_letter}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            cv2.imshow('Hand Tracking', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# GUI function to train and test the model
def gui():
    global train_dir, test_dir
    train_dir = r"C:\Users\17cha\Downloads\asl\asl_alphabet_train"
    test_dir = r"C:\Users\17cha\Downloads\asl\asl_alphabet_test"

    def start_training():
        global model, class_names
        model, class_names = train_model(train_dir, test_dir)

    def start_testing():
        hand_tracking(model, class_names)

    # Setup GUI
    window = tk.Tk()
    window.title("ASL Recognition")

    train_button = tk.Button(window, text="Train Model", command=start_training)
    train_button.pack(pady=10)

    test_button = tk.Button(window, text="Start Testing (Camera)", command=start_testing)
    test_button.pack(pady=10)

    window.mainloop()

if __name__ == "__main__":
    model = None
    class_names = None
    gui()


Found 87000 images belonging to 29 classes.
Found 29 images belonging to 29 classes.
Train samples: 87000
Test samples: 29
Epoch 1/2
2719/2719 [==============================] - 297s 109ms/step - loss: 1.6782 - accuracy: 0.4673
Epoch 2/2
1/1 [==============================] - 0s 227ms/step - loss: 0.0431 - accuracy: 1.0000
Test accuracy: 100.00%
1/1 [==============================] - 0s 16ms/step
